In [17]:
!pip install pyarrow duckdb pandas numpy -q

# 📦 Instalação de Pacotes

Antes de começar, vamos instalar os pacotes necessários:

# Capitulo 10 Casos Uso Otimizacoes

Notebook gerado automaticamente a partir do código fonte python.


## 📚 Importação de Bibliotecas

Importando as bibliotecas necessárias para o capítulo:

In [18]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import duckdb
import pandas as pd
import numpy as np
import os

# Configurações de exibição
pd.options.display.max_columns = None
print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


In [19]:
print(f"\n{'='*80}")
print(f"--- {'BEST PRACTICES AVANÇADAS: Production-Ready Patterns'.upper()} ---")
print(f"{'='*80}\n")

# Melhores práticas para ambientes de produção

# ==== 1. MEMORY MANAGEMENT ====
print("💾 FASE 1: Gerenciamento de Memória e Recursos\n")

# 1.1 Configuração otimizada do DuckDB
con_prod = duckdb.connect(':memory:')  # In-memory para performance
con_prod.execute("SET threads TO 4")
con_prod.execute("SET memory_limit='1GB'")
con_prod.execute("SET temp_directory='data_output/temp'")
con_prod.execute("SET preserve_insertion_order=false")  # Mais rápido

print("  ✓ Configurações de produção aplicadas:")
configs = con_prod.execute("""
    SELECT name, value 
    FROM duckdb_settings() 
    WHERE name IN ('threads', 'memory_limit', 'temp_directory', 'preserve_insertion_order')
""").fetchdf()
print(configs.to_string(index=False))

# 1.2 Zero-Copy com Arrow (registrar tabelas sem copiar)
print("\n  → Registrando tabelas com Zero-Copy:")
start_register = time.time()
con_prod.register('vendas_view', data)
register_time = time.time() - start_register
print(f"    ✓ Tabela registrada em {register_time:.4f}s (zero-copy, sem duplicação)")

# ==== 2. BATCH PROCESSING ====
print(f"\n⚙️ FASE 2: Processamento em Batches para Grandes Volumes\n")

def process_in_batches(table, batch_size=10000, operation_func=None):
    """Processa uma Arrow Table em batches"""
    results = []
    num_batches = (table.num_rows + batch_size - 1) // batch_size
    
    print(f"  → Processando {table.num_rows:,} registros em {num_batches} batches de {batch_size:,}")
    
    for i in range(0, table.num_rows, batch_size):
        batch_data = table.slice(i, min(batch_size, table.num_rows - i))
        
        # Aplicar operação customizada
        if operation_func:
            result = operation_func(batch_data, i // batch_size + 1)
            results.append(result)
    
    return results

# Exemplo: processar em batches
def batch_aggregation(batch_table, batch_num):
    """Agregação customizada por batch"""
    con_prod.register(f'batch_{batch_num}', batch_table)
    agg = con_prod.execute(f"""
        SELECT 
            {batch_num} as batch_id,
            COUNT(*) as registros,
            ROUND(AVG(preco_unitario), 2) as preco_medio,
            SUM(quantidade) as qtd_total
        FROM batch_{batch_num}
    """).fetch_arrow_table()
    con_prod.unregister(f'batch_{batch_num}')  # Liberar memória
    return agg

start_batch = time.time()
batch_results = process_in_batches(data, batch_size=10000, operation_func=batch_aggregation)
batch_time = time.time() - start_batch

# Consolidar resultados
con_prod.register('all_batches', pa.concat_tables(batch_results))
consolidated = con_prod.execute("""
    SELECT 
        COUNT(DISTINCT batch_id) as total_batches,
        SUM(registros) as total_registros,
        ROUND(AVG(preco_medio), 2) as preco_medio_geral,
        SUM(qtd_total) as qtd_total_geral
    FROM all_batches
""").fetch_arrow_table()

print(f"  ✓ Processamento em batches concluído em {batch_time:.4f}s")
print("\n  → Resultado Consolidado:")
print(consolidated.to_pandas().to_string(index=False))

# ==== 3. ERROR HANDLING E VALIDAÇÃO ====
print(f"\n🛡️ FASE 3: Error Handling Robusto e Validação de Dados\n")

def safe_query_execution(connection, query, query_name="Query"):
    """Execução segura com error handling"""
    try:
        start = time.time()
        result = connection.execute(query).fetch_arrow_table()
        elapsed = time.time() - start
        
        # Validações
        if result.num_rows == 0:
            print(f"  ⚠️ {query_name}: Retornou 0 linhas")
            return None
        
        print(f"  ✓ {query_name}: {result.num_rows:,} linhas em {elapsed:.4f}s")
        return result
        
    except Exception as e:
        print(f"  ✗ {query_name}: ERRO - {str(e)[:100]}")
        return None

# Testar com queries válidas e inválidas
print("  → Testando queries com error handling:")

# Query válida
valid_query = "SELECT regiao, COUNT(*) as total FROM vendas_view GROUP BY regiao"
result1 = safe_query_execution(con_prod, valid_query, "Query Válida")

# Query com tabela inexistente (gera erro)
invalid_query = "SELECT * FROM tabela_nao_existe"
result2 = safe_query_execution(con_prod, invalid_query, "Query Inválida")

# Query que retorna vazio
empty_query = "SELECT * FROM vendas_view WHERE 1=0"
result3 = safe_query_execution(con_prod, empty_query, "Query Vazia")

# ==== 4. SCHEMA VALIDATION ====
print(f"\n📐 FASE 4: Validação de Schema e Tipos\n")

def validate_schema(table, expected_schema):
    """Valida se a tabela tem o schema esperado"""
    issues = []
    
    # Verificar colunas
    expected_cols = set(expected_schema.names)
    actual_cols = set(table.schema.names)
    
    missing = expected_cols - actual_cols
    extra = actual_cols - expected_cols
    
    if missing:
        issues.append(f"Colunas faltando: {missing}")
    if extra:
        issues.append(f"Colunas extras: {extra}")
    
    # Verificar tipos das colunas comuns
    for col in expected_cols.intersection(actual_cols):
        expected_type = expected_schema.field(col).type
        actual_type = table.schema.field(col).type
        if expected_type != actual_type:
            issues.append(f"Coluna '{col}': tipo esperado {expected_type}, obtido {actual_type}")
    
    return issues

# Definir schema esperado
expected_schema = pa.schema([
    ('id', pa.int64()),
    ('data_venda', pa.timestamp('ns')),
    ('produto', pa.string()),
    ('quantidade', pa.int64()),
    ('preco_unitario', pa.float64())
])

print("  → Validando schema da tabela de vendas:")
validation_issues = validate_schema(data, expected_schema)
if validation_issues:
    print("  ⚠️ Problemas encontrados:")
    for issue in validation_issues:
        print(f"    • {issue}")
else:
    print("  ✓ Schema válido!")

# ==== 5. CONNECTION POOLING E REUSO ====
print(f"\n🔄 FASE 5: Connection Pooling e Reuso de Recursos\n")

class DuckDBConnectionPool:
    """Pool simples de conexões DuckDB"""
    def __init__(self, pool_size=3):
        self.pool = [duckdb.connect(':memory:') for _ in range(pool_size)]
        self.available = list(range(pool_size))
        self.in_use = {}
    
    def acquire(self):
        if self.available:
            conn_id = self.available.pop(0)
            self.in_use[conn_id] = self.pool[conn_id]
            return conn_id, self.pool[conn_id]
        return None, None
    
    def release(self, conn_id):
        if conn_id in self.in_use:
            del self.in_use[conn_id]
            self.available.append(conn_id)
    
    def stats(self):
        return {
            'total': len(self.pool),
            'available': len(self.available),
            'in_use': len(self.in_use)
        }

# Criar pool
pool = DuckDBConnectionPool(pool_size=3)
print(f"  ✓ Connection pool criado: {pool.stats()}")

# Simular uso do pool
print("\n  → Simulando queries concorrentes com pool:")
for i in range(5):
    conn_id, conn = pool.acquire()
    if conn:
        conn.register('data_temp', data)
        result = conn.execute("SELECT COUNT(*) as total FROM data_temp").fetchone()
        print(f"    • Query {i+1}: conexão #{conn_id}, resultado: {result[0]:,} registros")
        pool.release(conn_id)
    else:
        print(f"    • Query {i+1}: aguardando conexão disponível...")

print(f"  ✓ Estado final do pool: {pool.stats()}")

# ==== 6. MONITORING E PROFILING ====
print(f"\n📊 FASE 6: Monitoring e Profiling de Performance\n")

# Habilitar profiling detalhado
con_prod.execute("PRAGMA enable_profiling='json'")
con_prod.execute("PRAGMA profiling_output='data_output/profile.json'")

monitoring_query = """
    SELECT 
        regiao,
        produto,
        COUNT(*) as vendas,
        SUM(quantidade * preco_unitario) as receita
    FROM vendas_view
    WHERE status = 'Completado'
    GROUP BY regiao, produto
    HAVING vendas > 100
    ORDER BY receita DESC
    LIMIT 20
"""

print("  → Executando query com profiling habilitado:")
start_profile = time.time()
profile_result = con_prod.execute(monitoring_query).fetch_arrow_table()
profile_time = time.time() - start_profile

print(f"    ✓ Query executada em {profile_time:.4f}s")
print(f"    ✓ Resultados: {profile_result.num_rows} linhas")
print(f"    ✓ Profile salvo em: data_output/profile.json")

# Obter estatísticas de memória
memory_stats = con_prod.execute("""
    SELECT 
        database_size as db_size,
        block_size,
        total_blocks,
        used_blocks,
        free_blocks
    FROM pragma_database_size()
""").fetchdf()

print("\n  → Estatísticas de Memória do DuckDB:")
print(memory_stats.to_string(index=False))

# ==== 7. RESUMO FINAL ====
print(f"\n{'='*80}")
print(f"✨ BEST PRACTICES - RESUMO")
print(f"{'='*80}")
print(f"  ✓ Memory Management: Configurações otimizadas aplicadas")
print(f"  ✓ Zero-Copy: Tabelas registradas sem duplicação")
print(f"  ✓ Batch Processing: {data.num_rows:,} registros em batches de 10k")
print(f"  ✓ Error Handling: Validação robusta implementada")
print(f"  ✓ Schema Validation: Verificação de tipos e colunas")
print(f"  ✓ Connection Pooling: Pool de 3 conexões gerenciado")
print(f"  ✓ Monitoring: Profiling e estatísticas habilitados")
print(f"{'='*80}")

print("\n" + "="*80)
print(f"🎯 FIM DO CAPÍTULO 10: CASOS DE USO E OTIMIZAÇÕES")
print("="*80)
print("\n📚 Tópicos Cobertos:")
print("  1. ETL Pipeline Avançado (Extract, Transform, Load)")
print("  2. Data Lake Architecture Multi-Nível")
print("  3. Incremental Loading com CDC e Merge/Upsert")
print("  4. Query Optimization com Window Functions e JOINs")
print("  5. Best Practices para Produção")
print("\n✅ Todas as implementações foram executadas com sucesso!")
print("="*80 + "\n")


--- BEST PRACTICES AVANÇADAS: PRODUCTION-READY PATTERNS ---

💾 FASE 1: Gerenciamento de Memória e Recursos

  ✓ Configurações de produção aplicadas:
                    name            value
            memory_limit        953.6 MiB
preserve_insertion_order            false
          temp_directory data_output/temp
                 threads                4

  → Registrando tabelas com Zero-Copy:
    ✓ Tabela registrada em 0.0000s (zero-copy, sem duplicação)

⚙️ FASE 2: Processamento em Batches para Grandes Volumes

  → Processando 50,000 registros em 5 batches de 10,000
  ✓ Processamento em batches concluído em 0.0174s

  → Resultado Consolidado:
 total_batches total_registros  preco_medio_geral qtd_total_geral
             5           50000             2520.1          500223

🛡️ FASE 3: Error Handling Robusto e Validação de Dados

  → Testando queries com error handling:
  ✓ Query Válida: 5 linhas em 0.0041s
  ✗ Query Inválida: ERRO - Catalog Error: Table with name tabela_nao_existe 

## 5️⃣ Best Practices

Melhores práticas no uso de Arrow e DuckDB:

In [20]:
print(f"\n{'='*80}")
print(f"--- {'OTIMIZAÇÃO AVANÇADA DE QUERIES'.upper()} ---")
print(f"{'='*80}\n")

# Técnicas avançadas de otimização com comparativos de performance

# ==== 1. BENCHMARK SETUP ====
print("⚙️ FASE 1: Setup de Benchmark")

# Registrar dados para queries
con.register('vendas', data)
print(f"  ✓ Dataset registrado: {data.num_rows:,} vendas")
print(f"  ✓ Colunas: {data.num_columns}")

# ==== 2. QUERY NÃO OTIMIZADA VS OTIMIZADA ====
print(f"\n🔍 FASE 2: Comparativo - Query Não Otimizada vs Otimizada\n")

# Query Não Otimizada (sem índices, sem filtros early, sem projection pushdown)
print("  ❌ Query NÃO OTIMIZADA:")
query_nao_otimizada = """
    SELECT * 
    FROM vendas
"""
start = time.time()
result_full = con.execute(query_nao_otimizada).fetch_arrow_table()
# Aplicar filtros e agregações em Python (ineficiente)
df_temp = result_full.to_pandas()
df_filtered = df_temp[df_temp['status'] == 'Completado']
resultado_nao_opt = df_filtered.groupby('produto').agg({
    'quantidade': 'sum',
    'preco_unitario': 'mean'
}).reset_index()
time_nao_otimizada = time.time() - start
print(f"    • Tempo: {time_nao_otimizada:.4f}s")
print(f"    • Registros processados: {result_full.num_rows:,}")
print(f"    • Técnica: Full table scan + Python aggregation")

# Query Otimizada (com projection, filter pushdown, agregação no DuckDB)
print("\n  ✅ Query OTIMIZADA:")
query_otimizada = """
    SELECT 
        produto,
        SUM(quantidade) as quantidade,
        ROUND(AVG(preco_unitario), 2) as preco_unitario
    FROM vendas
    WHERE status = 'Completado'
    GROUP BY produto
"""
start = time.time()
resultado_otimizado = con.execute(query_otimizada).fetch_arrow_table()
time_otimizada = time.time() - start
print(f"    • Tempo: {time_otimizada:.4f}s")
print(f"    • Speedup: {time_nao_otimizada/time_otimizada:.2f}x mais rápido")
print(f"    • Técnicas: Filter pushdown + Projection pushdown + SQL aggregation")

print(f"\n  📊 Melhoria de Performance: {((time_nao_otimizada - time_otimizada) / time_nao_otimizada * 100):.1f}%")

# ==== 3. EXPLAIN PLANS DETALHADOS ====
print(f"\n📋 FASE 3: Análise de Planos de Execução\n")

# Query complexa para análise
complex_query = """
    WITH vendas_por_vendedor AS (
        SELECT 
            vendedor_id,
            produto,
            regiao,
            COUNT(*) as total_vendas,
            SUM(quantidade * preco_unitario * (1 - desconto_percentual/100.0)) as receita
        FROM vendas
        WHERE status = 'Completado'
            AND data_venda >= '2023-06-01'
        GROUP BY vendedor_id, produto, regiao
    ),
    ranking_vendedores AS (
        SELECT 
            vendedor_id,
            SUM(receita) as receita_total,
            COUNT(DISTINCT produto) as produtos_vendidos,
            RANK() OVER (ORDER BY SUM(receita) DESC) as rank_receita
        FROM vendas_por_vendedor
        GROUP BY vendedor_id
    )
    SELECT 
        v.vendedor_id,
        v.produto,
        v.regiao,
        v.total_vendas,
        ROUND(v.receita, 2) as receita,
        r.receita_total,
        r.rank_receita
    FROM vendas_por_vendedor v
    JOIN ranking_vendedores r ON v.vendedor_id = r.vendedor_id
    WHERE r.rank_receita <= 10
    ORDER BY r.rank_receita, v.receita DESC
"""

print("  → EXPLAIN (Plano Lógico):")
explain_result = con.execute(f"EXPLAIN {complex_query}").fetchdf()
print(explain_result['explain_value'].iloc[0][:500] + "...")

print("\n  → EXPLAIN ANALYZE (Plano Físico com Métricas):")
start_analyze = time.time()
analyze_result = con.execute(f"EXPLAIN ANALYZE {complex_query}").fetchdf()
analyze_time = time.time() - start_analyze
print(f"    • Tempo total de execução: {analyze_time:.4f}s")
print("\n" + analyze_result['explain_value'].iloc[0][:800] + "...")

# Executar a query para ver resultados
actual_result = con.execute(complex_query).fetch_arrow_table()
print(f"\n  → Resultados: {actual_result.num_rows} linhas retornadas")

# ==== 4. WINDOW FUNCTIONS E OTIMIZAÇÕES ====
print(f"\n🪟 FASE 4: Window Functions Avançadas\n")

window_query = """
    SELECT 
        produto,
        regiao,
        DATE_TRUNC('month', data_venda) as mes,
        SUM(quantidade * preco_unitario * (1 - desconto_percentual/100.0)) as receita_mensal,
        -- Moving average (média móvel de 3 meses)
        ROUND(AVG(SUM(quantidade * preco_unitario * (1 - desconto_percentual/100.0))) 
            OVER (PARTITION BY produto, regiao 
                  ORDER BY DATE_TRUNC('month', data_venda) 
                  ROWS BETWEEN 2 PRECEDING AND CURRENT ROW), 2) as media_movel_3m,
        -- Rank dentro de cada região
        RANK() OVER (PARTITION BY regiao, DATE_TRUNC('month', data_venda) 
                     ORDER BY SUM(quantidade * preco_unitario * (1 - desconto_percentual/100.0)) DESC) as rank_na_regiao,
        -- Percentual do total da região
        ROUND(SUM(quantidade * preco_unitario * (1 - desconto_percentual/100.0)) * 100.0 / 
              SUM(SUM(quantidade * preco_unitario * (1 - desconto_percentual/100.0))) 
                OVER (PARTITION BY regiao, DATE_TRUNC('month', data_venda)), 2) as pct_da_regiao
    FROM vendas
    WHERE status = 'Completado'
    GROUP BY produto, regiao, DATE_TRUNC('month', data_venda)
    ORDER BY mes DESC, regiao, receita_mensal DESC
"""

start_window = time.time()
window_result = con.execute(window_query).fetch_arrow_table()
window_time = time.time() - start_window

print(f"  ✓ Query com Window Functions executada em: {window_time:.4f}s")
print(f"  ✓ Resultados: {window_result.num_rows:,} linhas")
print("\n  → Amostra dos Resultados (Top 10):")
print(window_result.to_pandas().head(10).to_string(index=False))

# ==== 5. JOINS COMPLEXOS E OTIMIZAÇÃO ====
print(f"\n🔗 FASE 5: Otimização de JOINs Complexos\n")

# Criar dimensão de produtos para JOIN
dim_produtos = pa.table({
    'produto': ['Notebook', 'Mouse', 'Teclado', 'Monitor', 'Webcam', 'Headset', 'SSD', 'RAM'],
    'categoria': ['Computadores', 'Acessórios', 'Acessórios', 'Periféricos', 
                  'Acessórios', 'Acessórios', 'Componentes', 'Componentes'],
    'margem_percentual': [15.0, 40.0, 35.0, 20.0, 30.0, 35.0, 25.0, 30.0]
})
con.register('dim_produtos', dim_produtos)

# JOIN Otimizado
join_query = """
    SELECT 
        p.categoria,
        v.regiao,
        COUNT(*) as total_vendas,
        SUM(v.quantidade) as qtd_total,
        ROUND(SUM(v.quantidade * v.preco_unitario * (1 - v.desconto_percentual/100.0)), 2) as receita_bruta,
        ROUND(SUM(v.quantidade * v.preco_unitario * (1 - v.desconto_percentual/100.0) * p.margem_percentual / 100.0), 2) as margem_lucro
    FROM vendas v
    INNER JOIN dim_produtos p ON v.produto = p.produto
    WHERE v.status = 'Completado'
    GROUP BY p.categoria, v.regiao
    ORDER BY margem_lucro DESC
"""

start_join = time.time()
join_result = con.execute(join_query).fetch_arrow_table()
join_time = time.time() - start_join

print(f"  ✓ JOIN executado em: {join_time:.4f}s")
print(f"  ✓ Tipo: INNER JOIN com agregações")
print("\n  → Análise de Margem por Categoria e Região:")
print(join_result.to_pandas().to_string(index=False))

# ==== 6. PERFORMANCE SUMMARY ====
print(f"\n{'='*80}")
print(f"⚡ RESUMO DE PERFORMANCE - OTIMIZAÇÕES")
print(f"{'='*80}")
print(f"  Query Não Otimizada: {time_nao_otimizada:.4f}s")
print(f"  Query Otimizada: {time_otimizada:.4f}s")
print(f"  Speedup: {time_nao_otimizada/time_otimizada:.2f}x")
print(f"  Query Complexa (CTEs + Window): {analyze_time:.4f}s")
print(f"  Window Functions: {window_time:.4f}s")
print(f"  JOIN com Agregações: {join_time:.4f}s")
print(f"  Total de registros processados: {data.num_rows:,}")
print(f"{'='*80}\n")


--- OTIMIZAÇÃO AVANÇADA DE QUERIES ---

⚙️ FASE 1: Setup de Benchmark
  ✓ Dataset registrado: 50,000 vendas
  ✓ Colunas: 11

🔍 FASE 2: Comparativo - Query Não Otimizada vs Otimizada

  ❌ Query NÃO OTIMIZADA:
    • Tempo: 0.0357s
    • Registros processados: 50,000
    • Técnica: Full table scan + Python aggregation

  ✅ Query OTIMIZADA:
    • Tempo: 0.0111s
    • Speedup: 3.23x mais rápido
    • Técnicas: Filter pushdown + Projection pushdown + SQL aggregation

  📊 Melhoria de Performance: 69.0%

📋 FASE 3: Análise de Planos de Execução

  → EXPLAIN (Plano Lógico):
┌───────────────────────────┐
│            CTE            │
│    ────────────────────   │
│         CTE Name:         │
│    vendas_por_vendedor    │
│                           ├──────────────┐
│       Table Index: 0      │              │
│                           │              │
│           ~1 row          │              │
└─────────────┬─────────────┘              │
┌─────────────┴─────────────┐┌─────────────┴─────────

## 4️⃣ Query Optimization

Otimizando queries para melhor desempenho:

In [21]:
print(f"\n{'='*80}")
print(f"--- {'INCREMENTAL LOADING AVANÇADO: CDC & MERGE/UPSERT'.upper()} ---")
print(f"{'='*80}\n")

# Implementação de Carregamento Incremental com Change Data Capture (CDC) e Merge/Upsert

# ==== 1. SETUP: ESTADO INICIAL DO DATA WAREHOUSE ====
print("🗄️ FASE 1: Setup - Estado Inicial do Data Warehouse")
incremental_base = "data_output/incremental_warehouse"
if os.path.exists(incremental_base):
    import shutil
    shutil.rmtree(incremental_base)
os.makedirs(incremental_base, exist_ok=True)

# Carga inicial (Snapshot T0)
print("  → Criando snapshot inicial (T0)...")
initial_snapshot = pa.table({
    'cliente_id': [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008],
    'nome': ['Ana Silva', 'Bruno Costa', 'Carla Lima', 'Daniel Santos', 
             'Elena Martins', 'Fernando Dias', 'Gabriela Rocha', 'Hugo Pinto'],
    'email': ['ana@email.com', 'bruno@email.com', 'carla@email.com', 'daniel@email.com',
              'elena@email.com', 'fernando@email.com', 'gabriela@email.com', 'hugo@email.com'],
    'status': ['Ativo', 'Ativo', 'Ativo', 'Ativo', 'Ativo', 'Ativo', 'Ativo', 'Ativo'],
    'saldo': [1500.50, 2300.00, 980.75, 4500.25, 1200.00, 3300.50, 890.00, 5100.75],
    'ultima_atualizacao': pd.to_datetime(['2023-01-01']*8),
    'versao': [1]*8
})

snapshot_path = f"{incremental_base}/clientes.parquet"
pq.write_table(initial_snapshot, snapshot_path)
print(f"    ✓ Snapshot inicial salvo: {initial_snapshot.num_rows} clientes")
print(f"    ✓ Path: {snapshot_path}\n")

# ==== 2. SIMULAÇÃO DE MUDANÇAS (CDC) ====
print("🔄 FASE 2: Change Data Capture - Detectando Alterações")

# Simular mudanças incrementais (T1)
# - Updates: clientes 1002, 1004, 1007
# - Inserts: novos clientes 1009, 1010
# - Deletes: cliente 1006 (soft delete)
print("  → Simulando batch incremental (T1)...")
incremental_changes = pa.table({
    'cliente_id': [1002, 1004, 1006, 1007, 1009, 1010],
    'nome': ['Bruno Costa', 'Daniel Santos', 'Fernando Dias', 'Gabriela Rocha', 
             'Isabela Ferreira', 'João Mendes'],
    'email': ['bruno.novo@email.com', 'daniel@email.com', 'fernando@email.com', 'gabriela@email.com',
              'isabela@email.com', 'joao@email.com'],
    'status': ['Ativo', 'Ativo', 'Inativo', 'Ativo', 'Ativo', 'Ativo'],  # 1006 soft deleted
    'saldo': [2500.00, 5000.00, 3300.50, 1200.00, 2100.00, 1800.50],  # saldos atualizados/novos
    'ultima_atualizacao': pd.to_datetime(['2023-02-01']*6),
    'versao': [2, 2, 2, 2, 1, 1]
})

changes_path = f"{incremental_base}/changes_t1.parquet"
pq.write_table(incremental_changes, changes_path)
print(f"    ✓ CDC capturado: {incremental_changes.num_rows} alterações")
print(f"    ✓ Tipo de operações: 3 UPDATEs + 2 INSERTs + 1 SOFT DELETE\n")

# ==== 3. MERGE/UPSERT OPERATION ====
print("🔀 FASE 3: Executando Merge/Upsert com DuckDB")

con.register('tabela_atual', pq.read_table(snapshot_path))
con.register('mudancas', incremental_changes)

# Estratégia de Merge: 
# 1. Identificar registros novos (INSERTs)
# 2. Atualizar registros existentes (UPDATEs)
# 3. Manter histórico de versões
start_merge = time.time()

merged_result = con.execute("""
    -- CTE para identificar última versão de cada cliente
    WITH ultima_versao AS (
        -- Combinar dados atuais e mudanças
        SELECT * FROM tabela_atual
        UNION ALL
        SELECT * FROM mudancas
    ),
    ranked AS (
        SELECT 
            *,
            ROW_NUMBER() OVER (PARTITION BY cliente_id ORDER BY ultima_atualizacao DESC, versao DESC) as rn
        FROM ultima_versao
    )
    SELECT 
        cliente_id,
        nome,
        email,
        status,
        saldo,
        ultima_atualizacao,
        versao
    FROM ranked
    WHERE rn = 1  -- Pegar apenas a versão mais recente
    ORDER BY cliente_id
""").fetch_arrow_table()

merge_time = time.time() - start_merge

# Salvar resultado do merge
merged_path = f"{incremental_base}/clientes_merged.parquet"
pq.write_table(merged_result, merged_path)
print(f"  ✓ Merge concluído em {merge_time:.3f}s")
print(f"  ✓ Total de clientes após merge: {merged_result.num_rows}")
print(f"  ✓ Resultado salvo em: {merged_path}\n")

# ==== 4. ANÁLISE DE MUDANÇAS ====
print("📊 FASE 4: Análise Detalhada das Mudanças")

# Comparar antes e depois
con.register('antes', initial_snapshot)
con.register('depois', merged_result)

analise = con.execute("""
    SELECT 
        'Inicial' as snapshot,
        COUNT(*) as total_clientes,
        COUNT(CASE WHEN status = 'Ativo' THEN 1 END) as clientes_ativos,
        ROUND(SUM(saldo), 2) as saldo_total
    FROM antes
    
    UNION ALL
    
    SELECT 
        'Pós-Merge' as snapshot,
        COUNT(*) as total_clientes,
        COUNT(CASE WHEN status = 'Ativo' THEN 1 END) as clientes_ativos,
        ROUND(SUM(saldo), 2) as saldo_total
    FROM depois
""").fetch_arrow_table()

print("  → Comparativo Antes vs Depois:")
print(analise.to_pandas().to_string(index=False))

# Detalhar mudanças específicas
print("\n  → Clientes com Mudanças Detectadas:")
mudancas_detalhadas = con.execute("""
    SELECT 
        d.cliente_id,
        a.nome as nome_anterior,
        d.nome as nome_atual,
        a.email as email_anterior,
        d.email as email_atual,
        a.saldo as saldo_anterior,
        d.saldo as saldo_atual,
        ROUND(d.saldo - a.saldo, 2) as diferenca_saldo,
        d.status as status_atual
    FROM antes a
    INNER JOIN depois d ON a.cliente_id = d.cliente_id
    WHERE a.email != d.email 
       OR a.saldo != d.saldo 
       OR a.status != d.status
""").fetch_arrow_table()
print(mudancas_detalhadas.to_pandas().to_string(index=False))

# Novos clientes
print("\n  → Novos Clientes Inseridos:")
novos = con.execute("""
    SELECT d.*
    FROM depois d
    LEFT JOIN antes a ON d.cliente_id = a.cliente_id
    WHERE a.cliente_id IS NULL
""").fetch_arrow_table()
print(novos.to_pandas().to_string(index=False))

# ==== 5. HISTÓRICO E AUDITORIA ====
print(f"\n📜 FASE 5: Mantendo Histórico Completo (SCD Type 2)")

# Salvando histórico completo com todas as versões
historico_path = f"{incremental_base}/historico_completo.parquet"
historico_completo = con.execute("""
    SELECT 
        *,
        CASE 
            WHEN cliente_id IN (SELECT cliente_id FROM mudancas) THEN 'MODIFICADO'
            ELSE 'INALTERADO'
        END as tipo_registro
    FROM (
        SELECT * FROM tabela_atual
        UNION ALL
        SELECT * FROM mudancas
    )
    ORDER BY cliente_id, versao
""").fetch_arrow_table()

pq.write_table(historico_completo, historico_path)
print(f"  ✓ Histórico completo salvo: {historico_completo.num_rows} registros")
print(f"  ✓ Path: {historico_path}")

# Visualizar histórico de um cliente específico
print("\n  → Histórico Completo do Cliente 1002 (Bruno):")
historico_bruno = con.execute("""
    SELECT * FROM historico_completo
    WHERE cliente_id = 1002
    ORDER BY versao
""").fetch_arrow_table()
print(historico_bruno.to_pandas().to_string(index=False))

# ==== 6. MÉTRICAS FINAIS ====
print(f"\n{'='*80}")
print(f"📈 MÉTRICAS DO PROCESSO INCREMENTAL")
print(f"{'='*80}")
print(f"  Snapshot Inicial: {initial_snapshot.num_rows} registros")
print(f"  Mudanças Detectadas (CDC): {incremental_changes.num_rows} registros")
print(f"  Estado Final: {merged_result.num_rows} registros")
print(f"  Tempo de Merge: {merge_time:.3f}s")
print(f"  Registros no Histórico: {historico_completo.num_rows}")
print(f"  Throughput: {initial_snapshot.num_rows / merge_time:,.0f} registros/segundo")
print(f"{'='*80}\n")


--- INCREMENTAL LOADING AVANÇADO: CDC & MERGE/UPSERT ---

🗄️ FASE 1: Setup - Estado Inicial do Data Warehouse
  → Criando snapshot inicial (T0)...
    ✓ Snapshot inicial salvo: 8 clientes
    ✓ Path: data_output/incremental_warehouse/clientes.parquet

🔄 FASE 2: Change Data Capture - Detectando Alterações
  → Simulando batch incremental (T1)...
    ✓ CDC capturado: 6 alterações
    ✓ Tipo de operações: 3 UPDATEs + 2 INSERTs + 1 SOFT DELETE

🔀 FASE 3: Executando Merge/Upsert com DuckDB
  ✓ Merge concluído em 0.005s
  ✓ Total de clientes após merge: 10
  ✓ Resultado salvo em: data_output/incremental_warehouse/clientes_merged.parquet

📊 FASE 4: Análise Detalhada das Mudanças
  → Comparativo Antes vs Depois:
 snapshot  total_clientes  clientes_ativos  saldo_total
  Inicial               8                8     19772.75
Pós-Merge              10                9     24683.00

  → Clientes com Mudanças Detectadas:
 cliente_id  nome_anterior     nome_atual     email_anterior          email_atu

## 3️⃣ Incremental Loading

Técnicas de carregamento incremental de dados:

In [22]:
print(f"\n{'='*80}")
print(f"--- {'DATA LAKE ARCHITECTURE AVANÇADA'.upper()} ---")
print(f"{'='*80}\n")

# Arquitetura Moderna de Data Lake com Particionamento Multi-Nível

# ==== 1. CRIAÇÃO DO DATA LAKE COM PARTICIONAMENTO HIERÁRQUICO ====
print("🏗️ FASE 1: Construindo Data Lake com Particionamento Multi-Nível")
lake_base = "data_output/data_lake_advanced"
import shutil
if os.path.exists(lake_base):
    shutil.rmtree(lake_base)
os.makedirs(lake_base, exist_ok=True)

# Adicionar colunas de particionamento temporal
start_partition = time.time()
con.register('vendas_raw', data)
vendas_partitioned = con.execute("""
    SELECT 
        *,
        YEAR(data_venda) as ano,
        MONTH(data_venda) as mes,
        DAY(data_venda) as dia
    FROM vendas_raw
    WHERE status = 'Completado'  -- Apenas vendas completadas no lake
""").fetch_arrow_table()

# Salvar com particionamento hierárquico: ano/mes/regiao
# Importante: ordem das colunas de partição afeta performance de queries
pq.write_to_dataset(
    vendas_partitioned,
    root_path=lake_base,
    partition_cols=['ano', 'mes', 'regiao'],
    basename_template='vendas-{i}.parquet',
    compression='zstd',  # Melhor compressão que snappy
    existing_data_behavior='overwrite_or_ignore'
)

partition_time = time.time() - start_partition
print(f"  ✓ Data Lake criado em: {lake_base}")
print(f"  ✓ Tempo de particionamento: {partition_time:.3f}s")

# Mostrar estrutura do Data Lake
print(f"\n📁 Estrutura do Data Lake (primeiras 20 pastas):")
count = 0
for root, dirs, files in os.walk(lake_base):
    if count >= 20:
        print("  ... (mais pastas)")
        break
    level = root.replace(lake_base, '').count(os.sep)
    indent = '  ' * level
    folder_name = os.path.basename(root)
    if folder_name:
        print(f"{indent}└─ {folder_name}/")
        count += 1

# ==== 2. METADATA E ESTATÍSTICAS DO DATA LAKE ====
print(f"\n📊 FASE 2: Analisando Metadata do Data Lake")

# Coletar estatísticas do dataset
dataset = ds.dataset(lake_base, format='parquet', partitioning='hive')
print(f"  ✓ Total de partições: {len(list(dataset.get_fragments()))}")
print(f"  ✓ Colunas disponíveis: {len(dataset.schema.names)}")
print(f"  ✓ Schema: {', '.join(dataset.schema.names[:10])}...")

# Calcular tamanho total dos arquivos
total_size = 0
file_count = 0
for root, dirs, files in os.walk(lake_base):
    for file in files:
        if file.endswith('.parquet'):
            filepath = os.path.join(root, file)
            total_size += os.path.getsize(filepath)
            file_count += 1

print(f"  ✓ Total de arquivos: {file_count}")
print(f"  ✓ Tamanho total: {total_size / (1024**2):.2f} MB")
print(f"  ✓ Tamanho médio por arquivo: {total_size / file_count / 1024:.2f} KB")

# ==== 3. QUERIES ANALÍTICAS COMPLEXAS ====
print(f"\n🔍 FASE 3: Executando Queries Analíticas Complexas")

# Query 1: Análise de vendas por região com partition pruning
print("\n  → Query 1: Top Produtos por Região (com partition pruning)")
start_q1 = time.time()
query1 = con.execute(f"""
    SELECT 
        regiao,
        produto,
        COUNT(*) as total_vendas,
        ROUND(SUM(preco_unitario * quantidade * (1 - desconto_percentual/100.0)), 2) as receita_total,
        ROUND(AVG(rating), 2) as rating_medio
    FROM read_parquet('{lake_base}/**/*.parquet', hive_partitioning=1)
    WHERE ano = 2023 AND mes >= 6  -- Partition pruning ativa
    GROUP BY regiao, produto
    ORDER BY receita_total DESC
    LIMIT 20
""").fetch_arrow_table()
q1_time = time.time() - start_q1
print(f"    ✓ Tempo: {q1_time:.3f}s | Resultados: {query1.num_rows} linhas")
print(query1.to_pandas().head(5).to_string(index=False))

# Query 2: Análise temporal com window functions
print("\n  → Query 2: Tendência de Vendas Mensal com Crescimento")
start_q2 = time.time()
query2 = con.execute(f"""
    WITH vendas_mensais AS (
        SELECT 
            ano,
            mes,
            regiao,
            COUNT(*) as total_vendas,
            ROUND(SUM(preco_unitario * quantidade * (1 - desconto_percentual/100.0)), 2) as receita
        FROM read_parquet('{lake_base}/**/*.parquet', hive_partitioning=1)
        GROUP BY ano, mes, regiao
    )
    SELECT 
        ano,
        mes,
        regiao,
        total_vendas,
        receita,
        LAG(receita) OVER (PARTITION BY regiao ORDER BY ano, mes) as receita_mes_anterior,
        ROUND(
            (receita - LAG(receita) OVER (PARTITION BY regiao ORDER BY ano, mes)) / 
            NULLIF(LAG(receita) OVER (PARTITION BY regiao ORDER BY ano, mes), 0) * 100, 
            2
        ) as crescimento_percentual
    FROM vendas_mensais
    ORDER BY regiao, ano, mes
""").fetch_arrow_table()
q2_time = time.time() - start_q2
print(f"    ✓ Tempo: {q2_time:.3f}s | Resultados: {query2.num_rows} linhas")
print(query2.to_pandas().head(10).to_string(index=False))

# Query 3: Análise de clientes com múltiplas agregações
print("\n  → Query 3: Análise de Comportamento de Clientes")
start_q3 = time.time()
query3 = con.execute(f"""
    SELECT 
        cliente_id,
        COUNT(DISTINCT produto) as produtos_distintos,
        COUNT(*) as total_compras,
        ROUND(SUM(preco_unitario * quantidade * (1 - desconto_percentual/100.0)), 2) as lifetime_value,
        ROUND(AVG(rating), 2) as rating_medio,
        MIN(data_venda) as primeira_compra,
        MAX(data_venda) as ultima_compra,
        DATEDIFF('day', MIN(data_venda), MAX(data_venda)) as dias_como_cliente
    FROM read_parquet('{lake_base}/**/*.parquet', hive_partitioning=1)
    GROUP BY cliente_id
    HAVING total_compras > 5  -- Apenas clientes recorrentes
    ORDER BY lifetime_value DESC
    LIMIT 15
""").fetch_arrow_table()
q3_time = time.time() - start_q3
print(f"    ✓ Tempo: {q3_time:.3f}s | Resultados: {query3.num_rows} linhas")
print(query3.to_pandas().to_string(index=False))

# ==== 4. PERFORMANCE SUMMARY ====
print(f"\n{'='*80}")
print(f"⚡ PERFORMANCE SUMMARY - DATA LAKE")
print(f"{'='*80}")
print(f"  Registros no Lake: {vendas_partitioned.num_rows:,}")
print(f"  Arquivos Parquet: {file_count}")
print(f"  Compressão: {total_size / (1024**2):.2f} MB (ratio: {vendas_partitioned.nbytes / total_size:.1f}x)")
print(f"  Query 1 (Partition Pruning): {q1_time:.3f}s")
print(f"  Query 2 (Window Functions): {q2_time:.3f}s")
print(f"  Query 3 (Complex Aggregations): {q3_time:.3f}s")
print(f"{'='*80}\n")


--- DATA LAKE ARCHITECTURE AVANÇADA ---

🏗️ FASE 1: Construindo Data Lake com Particionamento Multi-Nível
  ✓ Data Lake criado em: data_output/data_lake_advanced
  ✓ Tempo de particionamento: 0.069s

📁 Estrutura do Data Lake (primeiras 20 pastas):
└─ data_lake_advanced/
  └─ ano=2023/
    └─ mes=1/
      └─ regiao=Centro/
      └─ regiao=Leste/
      └─ regiao=Norte/
      └─ regiao=Oeste/
      └─ regiao=Sul/
    └─ mes=10/
      └─ regiao=Centro/
      └─ regiao=Leste/
      └─ regiao=Norte/
      └─ regiao=Oeste/
      └─ regiao=Sul/
    └─ mes=11/
      └─ regiao=Centro/
      └─ regiao=Leste/
      └─ regiao=Norte/
      └─ regiao=Oeste/
      └─ regiao=Sul/
  ... (mais pastas)

📊 FASE 2: Analisando Metadata do Data Lake
  ✓ Total de partições: 90
  ✓ Colunas disponíveis: 14
  ✓ Schema: id, data_venda, produto, quantidade, preco_unitario, desconto_percentual, status, cliente_id, vendedor_id, rating...
  ✓ Total de arquivos: 90
  ✓ Tamanho total: 1.01 MB
  ✓ Tamanho médio por arqu

## 2️⃣ Data Lake Architecture

Arquitetura de data lake com Arrow:

In [23]:
print(f"\n{'='*80}")
print(f"--- {'ETL PIPELINE AVANÇADO: Extract -> Transform -> Load'.upper()} ---")
print(f"{'='*80}\n")

# Pipeline ETL Completo e Avançado

# ==== EXTRACT ====
print("📥 FASE 1: EXTRACT")
start_extract = time.time()

# Simulando extração de múltiplas fontes
fonte_principal = data
print(f"  ✓ Fonte Principal: {fonte_principal.num_rows:,} registros extraídos")

# Criando tabela de metadados de produtos
metadados_produtos = pa.table({
    'produto': ['Notebook', 'Mouse', 'Teclado', 'Monitor', 'Webcam', 'Headset', 'SSD', 'RAM'],
    'categoria': ['Hardware', 'Periféricos', 'Periféricos', 'Hardware', 'Periféricos', 'Periféricos', 'Hardware', 'Hardware'],
    'peso_kg': [2.5, 0.1, 0.8, 5.0, 0.3, 0.4, 0.05, 0.02],
    'garantia_meses': [24, 12, 12, 36, 12, 12, 60, 60]
})
print(f"  ✓ Metadados de Produtos: {metadados_produtos.num_rows} registros")

extract_time = time.time() - start_extract
print(f"  ⏱ Tempo de extração: {extract_time:.3f}s\n")

# ==== TRANSFORM ====
print("🔄 FASE 2: TRANSFORM")
start_transform = time.time()

# Registrar tabelas no DuckDB para transformações complexas
con.register('vendas', fonte_principal)
con.register('produtos_meta', metadados_produtos)

# Transformação 1: Validação e Limpeza
print("  → Etapa 2.1: Validação e Limpeza de Dados")
dados_validados = con.execute("""
    SELECT 
        *,
        CASE 
            WHEN preco_unitario <= 0 THEN NULL
            WHEN quantidade <= 0 THEN NULL
            ELSE preco_unitario * quantidade * (1 - desconto_percentual/100.0)
        END as valor_total_venda
    FROM vendas
    WHERE status != 'Cancelado'  -- Remover vendas canceladas
        AND preco_unitario > 0
        AND quantidade > 0
""").fetch_arrow_table()
print(f"    ✓ Registros válidos: {dados_validados.num_rows:,} (removidos {fonte_principal.num_rows - dados_validados.num_rows:,})")

# Transformação 2: Enriquecimento com JOIN
print("  → Etapa 2.2: Enriquecimento com Metadados")
con.register('vendas_validadas', dados_validados)
dados_enriquecidos = con.execute("""
    SELECT 
        v.*,
        p.categoria,
        p.peso_kg,
        p.garantia_meses,
        v.quantidade * p.peso_kg as peso_total_kg
    FROM vendas_validadas v
    LEFT JOIN produtos_meta p ON v.produto = p.produto
""").fetch_arrow_table()
print(f"    ✓ Dados enriquecidos: {dados_enriquecidos.num_columns} colunas")

# Transformação 3: Agregações Complexas
print("  → Etapa 2.3: Agregações Analíticas")
con.register('vendas_enriquecidas', dados_enriquecidos)
agregacoes = con.execute("""
    WITH vendas_por_periodo AS (
        SELECT 
            DATE_TRUNC('month', data_venda) as mes,
            regiao,
            categoria,
            produto,
            COUNT(*) as total_vendas,
            SUM(valor_total_venda) as receita_total,
            SUM(quantidade) as qtd_vendida,
            AVG(rating) as rating_medio,
            COUNT(DISTINCT cliente_id) as clientes_unicos,
            COUNT(DISTINCT vendedor_id) as vendedores_ativos
        FROM vendas_enriquecidas
        GROUP BY 1, 2, 3, 4
    )
    SELECT 
        mes,
        regiao,
        categoria,
        produto,
        total_vendas,
        ROUND(receita_total, 2) as receita_total,
        qtd_vendida,
        ROUND(rating_medio, 2) as rating_medio,
        clientes_unicos,
        vendedores_ativos,
        ROUND(receita_total / total_vendas, 2) as ticket_medio
    FROM vendas_por_periodo
    ORDER BY mes DESC, receita_total DESC
""").fetch_arrow_table()
print(f"    ✓ Agregações geradas: {agregacoes.num_rows:,} linhas")

transform_time = time.time() - start_transform
print(f"  ⏱ Tempo de transformação: {transform_time:.3f}s\n")

# ==== LOAD ====
print("💾 FASE 3: LOAD")
start_load = time.time()

# Load em múltiplos formatos e destinos
output_base = "data_output/etl_advanced"
os.makedirs(output_base, exist_ok=True)

# 3.1: Dados validados em Parquet comprimido
validated_path = f"{output_base}/vendas_validadas.parquet"
pq.write_table(dados_validados, validated_path, compression='snappy')
print(f"  ✓ Dados validados: {validated_path}")

# 3.2: Dados enriquecidos particionados por região
enriched_path = f"{output_base}/vendas_enriquecidas"
pq.write_to_dataset(
    dados_enriquecidos, 
    root_path=enriched_path,
    partition_cols=['regiao', 'categoria'],
    compression='zstd'
)
print(f"  ✓ Dados enriquecidos (particionados): {enriched_path}")

# 3.3: Agregações finais
agg_path = f"{output_base}/agregacoes_mensais.parquet"
pq.write_table(agregacoes, agg_path, compression='gzip')
print(f"  ✓ Agregações mensais: {agg_path}")

load_time = time.time() - start_load
print(f"  ⏱ Tempo de load: {load_time:.3f}s\n")

# ==== MÉTRICAS FINAIS ====
total_time = extract_time + transform_time + load_time
print(f"{'='*80}")
print(f"📊 MÉTRICAS DO PIPELINE ETL")
print(f"{'='*80}")
print(f"  Registros Processados: {fonte_principal.num_rows:,}")
print(f"  Registros Validados: {dados_validados.num_rows:,} ({dados_validados.num_rows/fonte_principal.num_rows*100:.1f}%)")
print(f"  Registros Agregados: {agregacoes.num_rows:,}")
print(f"  Tempo Total: {total_time:.3f}s")
print(f"  Taxa de Processamento: {fonte_principal.num_rows/total_time:,.0f} registros/segundo")
print(f"{'='*80}\n")

# Visualizar amostra dos resultados
print("📋 Amostra das Agregações Mensais (Top 10 por Receita):")
print(agregacoes.to_pandas().head(10).to_string(index=False))


--- ETL PIPELINE AVANÇADO: EXTRACT -> TRANSFORM -> LOAD ---

📥 FASE 1: EXTRACT
  ✓ Fonte Principal: 50,000 registros extraídos
  ✓ Metadados de Produtos: 8 registros
  ⏱ Tempo de extração: 0.006s

🔄 FASE 2: TRANSFORM
  → Etapa 2.1: Validação e Limpeza de Dados
    ✓ Registros válidos: 44,964 (removidos 5,036)
  → Etapa 2.2: Enriquecimento com Metadados
    ✓ Dados enriquecidos: 16 colunas
  → Etapa 2.3: Agregações Analíticas
    ✓ Agregações geradas: 720 linhas
  ⏱ Tempo de transformação: 0.060s

💾 FASE 3: LOAD
  ✓ Dados validados: data_output/etl_advanced/vendas_validadas.parquet
  ✓ Dados enriquecidos (particionados): data_output/etl_advanced/vendas_enriquecidas
  ✓ Agregações mensais: data_output/etl_advanced/agregacoes_mensais.parquet
  ⏱ Tempo de load: 0.076s

📊 MÉTRICAS DO PIPELINE ETL
  Registros Processados: 50,000
  Registros Validados: 44,964 (89.9%)
  Registros Agregados: 720
  Tempo Total: 0.141s
  Taxa de Processamento: 355,495 registros/segundo

📋 Amostra das Agregações 

## 1️⃣ ETL Pipelines

Construindo pipelines ETL com Arrow e DuckDB:

In [24]:
# Dados de exemplo globais - Dataset Complexo para Casos de Uso Avançados
import time

try:
    print("\nGerando dados de exemplo complexos...")
    
    # Gerar dados de vendas realistas (50.000 registros)
    num_records = 50000
    np.random.seed(42)
    
    # Datas ao longo de 2 anos
    start_date = pd.Timestamp('2023-01-01')
    dates = pd.date_range(start_date, periods=num_records, freq='15min')
    
    # Produtos, Regiões, Status
    produtos = ['Notebook', 'Mouse', 'Teclado', 'Monitor', 'Webcam', 'Headset', 'SSD', 'RAM']
    regioes = ['Norte', 'Sul', 'Leste', 'Oeste', 'Centro']
    status = ['Completado', 'Pendente', 'Cancelado', 'Em Processamento']
    
    data = pa.table({
        'id': range(num_records),
        'data_venda': dates,
        'produto': np.random.choice(produtos, num_records),
        'regiao': np.random.choice(regioes, num_records),
        'quantidade': np.random.randint(1, 20, num_records),
        'preco_unitario': np.round(np.random.uniform(50.0, 5000.0, num_records), 2),
        'desconto_percentual': np.random.choice([0, 5, 10, 15, 20], num_records),
        'status': np.random.choice(status, num_records, p=[0.7, 0.15, 0.1, 0.05]),
        'cliente_id': np.random.randint(1000, 10000, num_records),
        'vendedor_id': np.random.randint(1, 100, num_records),
        'rating': np.round(np.random.uniform(1.0, 5.0, num_records), 1)
    })
    
    print(f"✓ Tabela PyArrow criada: {data.num_rows:,} linhas x {data.num_columns} colunas")
    print(f"✓ Schema: {', '.join([f.name for f in data.schema])}")
    print(f"✓ Tamanho em memória: ~{data.nbytes / (1024**2):.2f} MB")
    
except Exception as e:
    print(f"✗ Erro ao criar dados: {e}")

# Conexão DuckDB com configurações otimizadas
con = duckdb.connect()
con.execute("SET threads TO 4")
con.execute("SET memory_limit='2GB'")
print("✓ Conexão DuckDB estabelecida com configurações otimizadas")


Gerando dados de exemplo complexos...
✓ Tabela PyArrow criada: 50,000 linhas x 11 colunas
✓ Schema: id, data_venda, produto, regiao, quantidade, preco_unitario, desconto_percentual, status, cliente_id, vendedor_id, rating
✓ Tamanho em memória: ~4.03 MB
✓ Conexão DuckDB estabelecida com configurações otimizadas
